# Methods for Homework 2, Team 2
## DSC291 - Data Science for Scientists and Engineers

This notebook demonstrates how we use aws-jupyter to 
- automatically start EC2 instances of different types
- run the same timed script on all these instances
- retrieve the results from all instances
- and combine all results with pricing data to compare the performance of different instances

While this notebook does not describe our actual experiment (we're submitting another one for that), it is meant as a general desription of how it is possible to use aws-jupyter to run any such experiment. 

**Run the last cell to turn this notebook into a presentation!**

*Note: To successfully run this notebook, it's easiest to start it in the conda environment for packages availability.*

    $conda activate
$jupyter notebook


In [ ]:
# import stuff
from os.path import isfile,isdir
from os import mkdir,chdir,getcwd
import pickle as pk
import pandas as pd
import numpy as np
import subprocess
from time import sleep
import argparse
from IPython.display import clear_output

# enable inline graphics
%pylab inline  

### We start by listing the instances that are active on our account. 
This can be helpful, to monitor whether there are any unused instances that are still running, and costing us money.

(also, we still haven't been given access to https://ets-apps.ucsd.edu/dsc291-custom-aws/index.cgi, so this is our solution to try to make sure that our costs are not piling up...)

In [ ]:
!aws ec2 describe-instances \
    --query 'Reservations[*].Instances[*].{Type:InstanceType,\
                                           LaunchTime:LaunchTime,\
                                           Instance:InstanceId,\
                                           KeyName:KeyName,\
                                           State:State.Name,\
                                           Name:Tags[?Key==`cluster-name`]|[0].Value}' \
    --output table

### Here are a few options for stopping or terminating instances 
This one is just an example for if one notices any instances that seem to have been running for a while whithout usage. In that case we can uncomment a line and terminate/stop them manually by their instance ID. 

### Now let's import some data on which instance types are available, and how much they cost.
We will use a subset of the instanceType list to start multiple EC2 instances which we want to compare. We will use the pricing data later on, to compare performance to pricing. 

In [ ]:
pricedf = pd.read_csv('ec2ondemandprices20200419.csv')
pricedf

Here, we are making price column numeric. Then, for this initial test we select only the cheapest instances, to prevent spending too much money on something that might not work. 

In [ ]:
if pricedf['priceHour'].dtype == 'O':
    pricedf['priceHour'] = pd.to_numeric(pricedf.priceHour.str.extract('(\d*\.?\d.?\d.?\d.)')[0])

priceCutOff = 0.5
dfTest = pricedf[pricedf['priceHour'] < priceCutOff]

typeList = dfTest['instanceType'].tolist()
priceList = np.round(dfTest['priceHour'].tolist(),4)
print(typeList)
print(priceList)

### We'll have to set up each instance before running our test
Here, we are creating a script that we will run on each instance to set it up. (This is basically copy-pasted from the class materials.)

### Python wrapper for aws-jupyter
To use aws-jupyter effectively from a notebook, we are adding some code that makes it easier to run commands from here. 

### A test script to run on each instance

Here, we are defining a test script that we will be running on each instance. The calculations to be done are timed a hundred times, and the results are written to a pickle file. 

For this test, we are just running a quick and simple task (finding the 100,000th Fibonacci number recursively). For our actual experiment, this script is more involved. 

<font color=red> Add some detail here about what the FileGenerator script is doing </font>

In [ ]:
for typeL in typeList:
    
    # Create an instance of typeL
    print("Creating instance %s"%typeL)
    !aws-jupyter create -c 1 -t $typeL --name $typeL
    time.sleep(30) # wait for aws to configure the instance
    output = subprocess.Popen("aws-jupyter check --name %s"%typeL, shell=True, stdout=subprocess.PIPE).communicate()
    print("\t...done")
    
    ## The below directory should be the git repository. the remote directory "./" is "/home/ubuntu/hw2" on the AWS system
    check_file_send = subprocess.Popen('aws-jupyter send-dir --local /home/jakemdaly/Documents/courses/dsc-291/TeamGitRepo/Public-DSC291/hw2/ --remote ./', shell=True, stdout=subprocess.PIPE).communicate()
    print(f"Sent local directory to instance")
    
    # Now we need to find out what the IP address of the cluster is
    start_char = str(output).find("ubuntu")
    indexer = start_char
    while str(output)[indexer] != " ":
        indexer += 1
    remote_instance = str(output)[start_char:indexer]
    
    
    # NOTE: Access credentials  were removed from all scripts, so they will not appear down here. You won't be able to run this part of the notebook
    print(f"Remote instance: {remote_instance}")
    print("Running the file generator and measuring latencies...")
    check1 = subprocess.Popen("ssh -i /home/jakemdaly/.ssh/JakeTeam2KeyPair.pem %s 'pip install s3cmd; cd hw2; /home/ubuntu/.local/bin/s3cmd get s3://jakes-bucket-dsc-291/*.txt . --access_key= --secret_key= --region us-west-2 --force; /usr/bin/python3 TestLatencies.py'"%remote_instance, shell=True, stdout=subprocess.PIPE).communicate()
    print("\t...done")
    print("Copying .csv files to local directory and terminating notebook")
    check2 = subprocess.Popen("scp -i /home/jakemdaly/.ssh/JakeTeam2KeyPair.pem %s:/home/ubuntu/hw2/latencies.csv /home/jakemdaly/Documents/courses/dsc-291/TeamGitRepo/Public-DSC291/hw2/"%remote_instance, shell=True, stdout=subprocess.PIPE).communicate()
    check3 = subprocess.Popen("mv latencies.csv latencies_%s.csv"%(typeL), shell=True, stdout=subprocess.PIPE).communicate()
    check4 = subprocess.Popen("aws-jupyter terminate --name %s"%typeL, shell=True).communicate()

In [ ]:
# Make sure all the instances are closed.
for typeL in typeList:
    subprocess.call("aws-jupyter terminate --name %s"%typeL, shell=True)

### A test script to run on each instance

Here, we are defining a test script that we will be running on each instance. The calculations to be done are timed a hundred times, and the results are written to a pickle file. 

For this test, we are just running a quick and simple task (finding the 100,000th Fibonacci number recursively). For our actual experiment, this script is more involved. 

<font color=red> Add some detail here about what the FileGenerator script is doing </font>

### Before we start, let's check which instances we will actually be using.
We can also further make the list shorter here, if we are just trying to see if this notebook works.

In [ ]:
# if you want to just try, say, 3 different ones for testing
# typeList = typeList[0:3]
typeList

## Now lets start all instances in the list and run our timed experiment multiple times!
This is where things actually happen. 

<font color=red> Not sure if we need this code below. Ill uncomment and see if the main chunk runs </font>

### Now, get all the collected data back into this notebook
Here, we create a pandas dataframe with all the data collected from the different types of instances. We need to collect the data from all the different files that were sent back to us from the different instances we've created.

We can print the first few lines of the dataframe to see if the data looks about right. 

*Here, something must have gone wrong on the instance of type **c5.large** that we had created. The output logs (which were deleted because they contained our AWS credentials) seem to indicate that the ssh connection failed when trying to retrieve the file with the results.*

In [ ]:
# results = pd.DataFrame(columns = typeList)
# for thisType in typeList:
#     filename = 'from_remote/stats_' + thisType.replace('.','-') + '.pkl'
#     if isfile(filename):
#         infile = open(filename,'rb')
#         thisData = pk.load(infile)
#         infile.close()
#         results[thisType] = thisData

# # display the top of the dataframe
# results.head()

### Now, we can finally examine the data!
Here, we are just creating a boxplot of the execution times by instance type. We see that the (cheap) instances that we tested all needed on average between 0.12 to 0.24 seconds to determine the 100,00th Fibonacci number recursively. The fastest instance types here are the compute optimized **c5.xlarge** and **c5.2xlarge**, and the memory optimized **r5d.xlarge**. 

Not surprisingly, these faster instances were also some of the comparatively more expensive ones amongst our set of types. So is it actually worth it paying that higher price to cut the execution time down?

In [ ]:
# bp1 = results.boxplot(rot=90)
# bp1.set_title('execution times on different instance types')
# bp1.set_ylabel('execution time in seconds')

### Cost-effectiveness
Here, we multiply the time the computations took by the hourly price of the instance to see how much the computation actually cost us on each type of instance. *(the numbers are very low, since all prices were <$0.50/hr and computation times were less than a second...)*

In [ ]:
# byPrice = pd.DataFrame(columns = typeList)
# for i, thisType in enumerate(typeList):
#     byPrice[thisType] = priceList[i] * (results[thisType] / 3600)
# byPrice.head()

### Now let's see how adjusting by price changes our perception of instance performance!
Now we see that for the same task, <span style="color:red">the least powerful instance types were actually much more efficient</span> than the more powerful ones in terms of total cost!

So, if you have a lot of time and are really just interested in recursively calculating Fibonacci numbers, then you should do so on a **t3.nano** instance! 

We assume that this price effectiveness will change when you do more complex tasks, such as moving large files between S3 and EC2 instances. This is why the second notebook in our submission for homework 2 is testing this scenario, focussing more on the analysis of the results than on the methods of how to get things running. 

In [ ]:
# bp2 = byPrice.boxplot(rot=90)
# bp2.set_title('cost-effectiveness of different instance types')
# bp2.set_ylabel('price for execution time of the script')

In [ ]:
# # To view this notebook as slides
# !jupyter nbconvert hw2psa.ipynb --to slides --post serve